# Mongo Builder

You don't need to nor can you use this, it's internal to the DBA of the Mongo instance. 

Additionally, this is LEGACY: it generates off the PokeroleBot csv data. Those files aren't going to be updated any further.

This is mostly here for record keeping and reference

In [2]:
import yaml
import json
from pymongo import MongoClient
import os
import pandas as pd
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

secrets = json.load(open('../secrets.json'))

client = MongoClient(host='aurii.us', port=20180, username=secrets['MongoUser'], password=secrets['MongoPass'])


learnsets_path = '../raw/PokeroleBot/PokeLearnMovesFull.csv'
moves_path = '../raw/PokeroleBot/pokeMoveSorted.csv'
abilities_path = '../raw/PokeroleBot/PokeRoleAbilities.csv'
items_path = '../raw/PokeroleBot/PokeRoleItems.csv'
pokedex_path = '../raw/PokeroleBot/PokeroleStats.csv'
natures_path = '../raw/PokeroleBot/Natures.csv'

pokedex_output = '../Version20/Pokedex'
abilities_output = '../Version20/Abilities'
moves_output = '../Version20/Moves'
learnsets_output = '../Version20/Learnsets'
natures_output = '../Version20/Natures'
# json_output = '../Version20/JSON'

os.makedirs(pokedex_output,exist_ok=True)
os.makedirs(abilities_output,exist_ok=True)
os.makedirs(moves_output,exist_ok=True)
os.makedirs(learnsets_output,exist_ok=True)
os.makedirs(natures_output,exist_ok=True)
# os.makedirs(json_output,exist_ok=True)

/Users/bill/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def chunk_data(bulk, x=500):
    ret = []
    for i in range(0,len(bulk),x):
        ret.append(bulk[i:i+x])
    return ret

def publish(dataset, path, fname_key='', chunk_size=0, mongo=None):
    if mongo is not None: mongo.delete_many({})
    if chunk_size:
        chunks = []
        for i in range(0,len(dataset),chunk_size):
            chunks.append(dataset[i:i+chunk_size])
    else: chunks = dataset
    
    for i in range(len(chunks)):
        fname = chunks[i][fname_key] if fname_key else f"{os.path.basename(path)}{i}"
        open(path+f"/{fname}.json",'w').write(json.dumps(chunks[i], indent=4))
        if mongo is not None: 
            if type(chunks[i]) == list: mongo.insert_many(chunks[i])
            else: mongo.insert_one(chunks[i])

# Pokedex Entries

In [4]:
data = pd.read_csv(pokedex_path)
data.columns = data.columns.str.title().str.replace(' ','')
data = data.rename({'No.':'Number', 'Hp': 'BaseHP','Unevolved?':'Unevolved', 'Dexid':'DexID'},axis=1)
data = data.fillna('')
bulk_json = []
for i, row in data.iterrows():
    yam = row.to_dict()
    yam['_id'] = yam['Name']
    if yam['Unevolved']: yam['Unevolved'] = True
    if yam['HasAForm?']: yam['HasForm'] = True
    if yam['Legendary']: yam['Legendary'] = True
    if yam['GoodStarter']: yam['GoodStarter'] = True

    if not yam['Unevolved']: yam['Unevolved'] = False
    if not yam['HasAForm?']: yam['HasForm'] = False
    if not yam['Legendary']: yam['Legendary'] = False
    if not yam['GoodStarter']: yam['GoodStarter'] = False
    yam['Learnset'] = f"{yam['Name']}-Learnset"
    yam["Abilities"] = (f"{yam['Ability1']}"
                        f"{'' if not yam['Ability2'] else ' / '+ yam['Ability2']}"
                        f"{'' if not yam['HiddenAbility'] else ' ('+yam['HiddenAbility']+')'}"
                        f"{'' if not yam['EventAbilities'] else ' <'+yam['EventAbilities']+'>'}"
                        )
    yam["Type"] = f"{yam['Type1']} / {yam['Type2']}" if yam['Type2'] else yam['Type1']
    INTEGERS = ['BaseHP', 'Strength', 'MaxStrength',
       'Dexterity', 'MaxDexterity', 'Vitality', 'MaxVitality', 'Special',
       'MaxSpecial', 'Insight', 'MaxInsight']
    for key in INTEGERS:
        yam[key] = int(yam[key])
    del yam['HasAForm?']
    bulk_json.append(yam)

db = client['Pokerole20']
# publish(bulk_json, pokedex_output, chunk_size=500, mongo=db.Pokedex)
publish(bulk_json, pokedex_output, fname_key="Name", mongo=db.Pokedex)

# Ability Entries

In [7]:
data = pd.read_csv(abilities_path)
data = data.fillna('')
bulk_json = []
for i, row in data.iterrows():
    yam = row.to_dict()
    yam['_id'] = yam['Name']
    bulk_json.append(yam)
db = client['Pokerole20']
publish(bulk_json, abilities_output, fname_key="Name", mongo=db.Abilities)

# Move Dex



In [8]:
data = pd.read_csv(moves_path)
data = data.fillna('')
data.Type = data.Type.str.capitalize()
bulk_json = []
for i, row in data.iterrows():
    yam = row.to_dict()
    yam['_id'] = yam['Name']
    bulk_json.append(yam)
db = client['Pokerole20']
publish(bulk_json, moves_output, fname_key="Name", mongo=db.Moves)

# Move Lists

In [9]:
data = pd.read_csv(learnsets_path)
data = data.fillna('')
bulk_json = []
for i, row in data.iterrows():
    yam = {}
    yam['Name'] = row['Name']
    yam['_id'] = row['Name']
    yam['Species'] = f"{row['Name']}"
    yam['DexID'] = row['No.']
    moves = []
    for i in range(1,29):
        if row[f'Rank{i}']:
#             if row[f'Rank{i}'] not in moves:
#                 moves[row[f'Rank{i}']] = []
            m = row[f'Move{i}']
            moves.append({"Learned":row[f'Rank{i}'],"Name":row[f'Move{i}']})
    yam['Moves'] = moves
    bulk_json.append(yam)

db = client['Pokerole20']
publish(bulk_json, learnsets_output, fname_key="Name", mongo=db.Learnsets)

# Natures

In [10]:
data = pd.read_csv(natures_path,sep=',', skipinitialspace=True)
data = data.fillna('')
bulk_json = []
for i, row in data.iterrows():
    yam = row.to_dict()
    yam['_id'] = row['Name']
    bulk_json.append(yam)

db = client['Pokerole20']
publish(bulk_json, natures_output, fname_key="Name", mongo=db.Natures)

# Scratch